## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. Please push your code afterwards so I can share it with other students!

## Mac Users

I find some challenges while setting up this in MAC silicon M1 chip. Execute below commands in MAC terminal.

1. Download chromedriver.
2. Unzip and add it to the path.
3. Set Extended attributes.

cd  ~/Downloads
wget https://storage.googleapis.com/chrome-for-testing-public/133.0.6943.126/mac-arm64/chromedriver-mac-arm64.zip
unzip chromedriver-mac-arm64.zip
sudo mv chromedriver-mac-arm64/chromedriver /usr/local/bin/
chmod +x /usr/local/bin/chromedriver
cd /usr/local/bin/
xattr -d com.apple.quarantine chromedriver
cd 
chromedriver --version

In [1]:
!pip install selenium
!pip install undetected-chromedriver
!pip install beautifulsoup4

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from openai import OpenAI
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [7]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"
PATH_TO_CHROME_DRIVER = '/usr/local/bin/chromedriver'
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown. Highlight all the products this website offered and also find when website is created."


In [8]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service(PATH_TO_CHROME_DRIVER)
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # input("Please complete the verification in the browser and press Enter to continue...")
        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [5]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt



def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]


def summarize(url):
    website = Website(url)
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content


def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [9]:
display_summary("https://ae.almosafer.com")

It appears that you have provided a sample website or travel booking platform, specifically for flights and hotels in the Middle East region. The content includes:

1. **Flights**: A search engine to find flights across various airlines.
2. **Hotels**: A selection of chain hotels available for booking.
3. **Travel**: A general page with FAQs and information about traveling within Saudi Arabia, Kuwait, and other nearby countries.
4. **Almosafer App**: An advertisement for the Almosafer app, which offers features like secure payment channels, easy booking processes, and user-friendly designs.

The platform also displays a list of trending searches, airlines, and countries to facilitate searching and planning trips.

Please let me know if you have any specific questions or need further assistance with this website sample.